In [25]:
import cv2
import numpy as np
import plotly.graph_objects as go
from PIL import Image
import pillow_heif
pillow_heif.register_heif_opener()

import time
from IPython.display import display, clear_output

In [33]:
# Given metadata
focal_length_mm = 26
sensor_width_mm = 5.74
image_width_px = 3072
image_height_px = 2304

# Calculate focal length in pixels
fx = (focal_length_mm * image_width_px) / sensor_width_mm
fy = fx 
cx = image_width_px / 2
cy = image_height_px / 2

# Construct intrinsic matrix
K = np.array([[fx, 0, cx],
              [0, fy, cy],
              [0, 0, 1]])

print("Intrinsic Matrix K:\n", K)

Intrinsic Matrix K:
 [[1.39149826e+04 0.00000000e+00 1.53600000e+03]
 [0.00000000e+00 1.39149826e+04 1.15200000e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [34]:
# Load images
img1 = Image.open('../camera_stream/P1180308.JPG')
img2 = Image.open('../camera_stream/P1180309.JPG')

gray1 = cv2.cvtColor(np.array(img1), cv2.COLOR_RGB2GRAY)
gray2 = cv2.cvtColor(np.array(img2), cv2.COLOR_RGB2GRAY)

# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect keypoints and compute descriptors
kp1, des1 = sift.detectAndCompute(gray1, None)
kp2, des2 = sift.detectAndCompute(gray2, None)

# Use FLANN-based matcher for feature matching
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
search_params = dict(checks=50)

flann = cv2.FlannBasedMatcher(index_params, search_params)
# bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

# Perform knnMatch to get the two nearest neighbors
matches = flann.knnMatch(des1, des2, k=2)

# Apply Lowe's ratio test to find good matches
good_matches = []
pts1 = []
pts2 = []

for i, (m, n) in enumerate(matches):
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)
        pts1.append(kp1[m.queryIdx].pt)
        pts2.append(kp2[m.trainIdx].pt)

pts1 = np.asarray(pts1)
pts2 = np.asarray(pts2)

# Ensure there are enough matches
if len(good_matches) > 5:
    # Assume camera intrinsic parameters are known
    # For demonstration, we'll assume an identity matrix as intrinsic parameters
    # In practice, use actual camera calibration data
    K = np.identity(3)

    # Find the Essential matrix
    E, mask = cv2.findEssentialMat(pts1, pts2, K, method=cv2.RANSAC, prob=0.999, threshold=1.0)

    # Recover the relative camera pose from the Essential matrix
    _, R, t, mask_pose = cv2.recoverPose(E, pts1, pts2, K)

    # Construct projection matrices for each camera
    M1 = np.hstack((np.identity(3), np.zeros((3, 1))))  # First camera matrix: [I | 0]
    M2 = np.hstack((R, t))  # Second camera matrix: [R | t]

    # Convert points to homogeneous coordinates and normalize
    pts1_hom = cv2.convertPointsToHomogeneous(pts1)
    pts2_hom = cv2.convertPointsToHomogeneous(pts2)

    # Triangulate points to get 4D homogeneous coordinates
    pts4D_hom = cv2.triangulatePoints(M1, M2, pts1.T, pts2.T)

    # Convert to non-homogeneous 3D coordinates
    pts3D = pts4D_hom / pts4D_hom[3]
    pts3D = pts3D[:3].T  # Transpose to get Nx3 shape

    # Print the 3D points
    print(f"Recovered {len(good_matches)} 3D points.")

else:
    print("Not enough matches found between the two images.")

Recovered 13106 3D points.


In [35]:
# Draw matches
match_img = cv2.drawMatches(gray1, kp1, gray2, kp2, good_matches, None)

# Resize the image to fit within a smaller window
scale_percent = 20  # percent of original size
width = int(match_img.shape[1] * scale_percent / 100)
height = int(match_img.shape[0] * scale_percent / 100)
dim = (width, height)

# Resize image
resized_match_img = cv2.resize(match_img, dim, interpolation=cv2.INTER_AREA)

# Display matches
cv2.imshow("Matches", resized_match_img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [37]:
# Initialize the figure
fig = go.Figure()

min_ = -20
max_ = 20

fig.update_layout(
    scene=dict(
        xaxis=dict(
            title='X',
            range=[min_, max_] 
        ),
        yaxis=dict(
            title='Y',
            range=[min_, max_]  
        ),
        zaxis=dict(
            title='Z',
            range=[min_, max_] 
        )
    ),
    title="Interactive 3D SfM Model",
    width=800,  
    height=800 
)

# Set initial data
fig.add_trace(go.Scatter3d(
    x=[],
    y=[],
    z=[],
    mode='markers',
    marker=dict(size=1, color='red')
))

# Function to add points
def add_pointset(pts):
    # Generate random coordinates
    x = pts[:,0]
    y = pts[:,1]
    z = pts[:,2]

    # Append the new points to the existing data
    fig.data[0].x = list(fig.data[0].x) + list(x)
    fig.data[0].y = list(fig.data[0].y) + list(y)
    fig.data[0].z = list(fig.data[0].z) + list(z)


add_pointset(pts3D)
display(fig) 
 

In [ ]:
# Initialize SIFT detector
sift = cv2.SIFT_create()

# Detect key points and descriptors
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

# Match descriptors using FLANN matcher
index_params = dict(algorithm=1, trees=5)
search_params = dict(checks=50)
flann = cv2.FlannBasedMatcher(index_params, search_params)
matches = flann.knnMatch(des1, des2, k=2)

# Store all good matches as per Lowe's ratio test
good_matches = []
for m, n in matches:
    if m.distance < 0.7 * n.distance:
        good_matches.append(m)

# Draw matches
img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv2.imshow('Matches', img_matches)
cv2.waitKey(0)
cv2.destroyAllWindows()




# Extract matched keypoints
pts1 = np.float32([kp1[m.queryIdx].pt for m in good_matches])
pts2 = np.float32([kp2[m.trainIdx].pt for m in good_matches])

# Compute the fundamental matrix
F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC)

# We select only inlier points
pts1 = pts1[mask.ravel() == 1]
pts2 = pts2[mask.ravel() == 1]

# Compute the essential matrix
E = K.T @ F @ K

# Recover pose
_, R, t, mask = cv2.recoverPose(E, pts1, pts2, K)

# Triangulate points
pts1_hom = cv2.convertPointsToHomogeneous(pts1)[:, 0, :]
pts2_hom = cv2.convertPointsToHomogeneous(pts2)[:, 0, :]
P1 = np.hstack((np.eye(3), np.zeros((3, 1))))
P2 = np.hstack((R, t))
P1 = K @ P1
P2 = K @ P2

points_4D_hom = cv2.triangulatePoints(P1, P2, pts1.T, pts2.T)
points_3D = points_4D_hom[:3] / points_4D_hom[3]


# Prepare data for Plotly
x = points_3D[0]
y = points_3D[1]
z = points_3D[2]

# Create a 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=x,
    y=y,
    z=z,
    mode='markers',
    marker=dict(
        size=3,
        color=z,                # Set color to the Z values
        colorscale='Viridis',   # Choose a colorscale
        opacity=0.8
    )
)])

# Set plot layout
fig.update_layout(
    title='3D Point Cloud from SfM',
    scene=dict(
        xaxis_title='X Axis',
        yaxis_title='Y Axis',
        zaxis_title='Z Axis'
    )
)

# Show plot
fig.show()